# Aysnch Runs in LangGraph

* LG is very powerful, that is, the library can handle and help with aysnc running without packages like nest asyncio

In [15]:
from dotenv import load_dotenv

load_dotenv()


import operator
from typing import Annotated, Any

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END


class State(TypedDict):
    aggregate: Annotated[list, operator.add]


class ReturnNodeValue:
    def __init__(self, node_secret: str):
        self._value = node_secret # stores the value of a specific node, i.e. "I am"

    def __call__(self, state: State) -> Any:
        import time

        time.sleep(1)
        print(f"Adding {self._value} to {state['aggregate']}")
        return {"aggregate": [self._value]} # updates the long running annotated list

# Compile together
builder = StateGraph(State)
builder.add_node("a", ReturnNodeValue("I'm A"))
builder.add_edge(START, "a")
builder.add_node("b", ReturnNodeValue("I'm B"))
builder.add_node("b2", ReturnNodeValue("I'm B2"))
builder.add_node("c", ReturnNodeValue("I'm C"))
builder.add_node("d", ReturnNodeValue("I'm D"))
builder.add_edge("a", "b")
builder.add_edge("a", "c")
builder.add_edge("b", "b2")
builder.add_edge(["b2", "c"], "d")
builder.add_edge("d", END)
graph = builder.compile()
graph.get_graph().draw_mermaid_png(output_file_path="async.png")

if __name__ == "__main__":
    print("Helo Async Graph")
    # Like LangChain, you need to invoke for it to work.
    # graph.invoke({"aggregate": []}, {"configurable": {"thread_id": "foo"}})
    # Method 1: Print the config schema
    # print(graph.__dict__)
    # Start with basic state
    graph.invoke({"aggregate": []})
    # graph.invoke(
    # {"aggregate": []},  # State
    # {"configurable": {"thread_id": "123"}}  # Config
    # )


Helo Async Graph
Adding I'm A to []
Adding I'm B to ["I'm A"]
Adding I'm C to ["I'm A"]
Adding I'm B2 to ["I'm A", "I'm B", "I'm C"]
Adding I'm D to ["I'm A", "I'm B", "I'm C", "I'm B2"]


# Best Practices - Async
* Always test your async function
* Nodes should be writing to different attributes of the state to avoid override and conflicts with each other.
* Carefully map out what is happening async in your graph